#  DEEPLEARNING IMAGES

In [1]:
import glob
import pandas as pd
import tensorflow as tf


from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np

import os #Miscellaneous operating system interfaces
#https://docs.python.org/3/library/os.html
#get current working directory
path = os.getcwd() + '\\images\\image_train'
path

'C:\\Users\\Edgar\\Documents\\Rakuten\\images\\image_train'

Nombre de mots par texte:

## IMAGES

Recuperer les données images:

In [2]:
X = pd.read_csv('X_train_update.csv',index_col=0)
y = pd.read_csv('Y_train_CVw08PX.csv',index_col=0).squeeze().map(str)

#Create a column with the name of the picture
X['image_name'] = 'image_' + X['imageid'].map(str) + '_product_' + X['productid'].map(str) + '.jpg'
X['image_path'] = path + r'\image_' + X['imageid'].map(str) + '_product_' + X['productid'].map(str) + '.jpg'
print(X['image_name'].loc[0])
print(X['image_path'].loc[0])

image_1263597046_product_3804725264.jpg
C:\Users\Edgar\Documents\Rakuten\images\image_train\image_1263597046_product_3804725264.jpg


Concatener X_train et les labels:

In [3]:
X = pd.concat([X,y],axis=1)
X

,designation,description,productid,imageid,image_name,image_path,prdtypecode
0,Olivia: Personalisiertes Notizbuch / 150 Seite...,NaN,3804725264,1263597046,image_1263597046_product_3804725264.jpg,C:\Users\Edgar\Documents\Rakuten\images\image_...,10
1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,NaN,436067568,1008141237,image_1008141237_product_436067568.jpg,C:\Users\Edgar\Documents\Rakuten\images\image_...,2280
2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978,image_938777978_product_201115110.jpg,C:\Users\Edgar\Documents\Rakuten\images\image_...,50
3,Peluche Donald - Europe - Disneyland 2000 (Mar...,NaN,50418756,457047496,image_457047496_product_50418756.jpg,C:\Users\Edgar\Documents\Rakuten\images\image_...,1280
4,La Guerre Des Tuques,Luc a des id&eacute;es de grandeur. Il veut or...,278535884,1077757786,image_1077757786_product_278535884.jpg,C:\Users\Edgar\Documents\Rakuten\images\image_...,2705
...,...,...,...,...,...,...,...
84911,The Sims [ Import Anglais ],NaN,206719094,941495734,image_941495734_product_206719094.jpg,C:\Users\Edgar\Documents\Rakuten\images\image_...,40
84912,Kit piscine acier NEVADA déco pierre Ø 3.50m x...,<b>Description complète :</b><br />Kit piscine...,3065095706,1188462883,image_1188462883_product_3065095706.jpg,C:\Users\Edgar\Documents\Rakuten\images\image_...,2583
84913,Journal Officiel De La Republique Francaise N°...,NaN,440707564,1009325617,image_1009325617_product_440707564.jpg,C:\Users\Edgar\Documents\Rakuten\images\image_...,2280
84914,Table Basse Bois De Récupération Massif Base B...,<p>Cette table basse a un design unique et con...,3942400296,1267353403,image_1267353403_product_3942400296.jpg,C:\Users\Edgar\Documents\Rakuten\images\image_...,1560


Une fois nous avons un dataset de la taille desirée on peut le séparer en train et test sets:

In [4]:
X_train_img, X_test_img, y_train, y_test = train_test_split(X[['image_name','prdtypecode']], X.prdtypecode , train_size = 0.4, test_size=0.2, random_state=42)
#X_train_path, X_test_path, y_train, y_test = train_test_split(X.image_path, X.label, random_state=42)

In [5]:
X_train_img

,image_name,prdtypecode
60735,image_1208783386_product_2825941333.jpg,1320
9118,image_856119038_product_89102802.jpg,1281
55855,image_936925976_product_197015072.jpg,2403
42138,image_1166755995_product_2824252365.jpg,1302
10948,image_1017775450_product_418466190.jpg,10
...,...,...
76233,image_1075701044_product_1351876762.jpg,2583
47141,image_1174586088_product_2940638611.jpg,2583
42108,image_984906419_product_290034883.jpg,1140
72085,image_901400003_product_62981761.jpg,10


Transformation des données:

In [6]:
#APPLY SOME TRANSFORMATIONS TO DATA


train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)

train_set = train_datagen.flow_from_dataframe(dataframe=X_train_img,
                                              directory=path,
                                              x_col = "image_name",
                                              y_col = "prdtypecode",
                                              class_mode ="sparse",
                                              target_size = (224, 224),
                                              batch_size = 16)

test_set = test_datagen.flow_from_dataframe(dataframe=X_test_img,
                                              directory=path,
                                              x_col = "image_name",
                                              y_col = "prdtypecode",
                                              class_mode ="sparse",
                                              target_size = (224, 2244),
                                              batch_size = 16)

Found 33966 validated image filenames belonging to 27 classes.
Found 16984 validated image filenames belonging to 27 classes.


### MODELE POUR CLASSIFICATION D'IMAGE

In [7]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input, GlobalAveragePooling2D
from tensorflow.keras import Model

base_model = VGG16(weights='imagenet',input_shape=(224, 224, 3),include_top=False)

input_ = Input(shape=(224, 224, 3), name='image')
x = base_model(input_)
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)
#x = Flatten()(x) 
output = Dense(27, activation='softmax')(x)

model = Model(input_, output)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image (InputLayer)           [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 1024)              525312    
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0     

In [8]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [9]:
unfreezed_layers = 0 #Nombre de couches a décongeler pour aplique le finetuning: Voir livre Deep Learning with python
# Freezer les couches du VGG16
for layer in base_model.layers[-unfreezed_layers:]: 
    layer.trainable = False

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image (InputLayer)           [(None, 224, 224, 3)]     0         
_________________________________________________________________
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 1024)              525312    
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0     

In [11]:
#model.layers[1].trainable = False

In [12]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

Callbacks:

In [19]:
from tensorflow.keras import callbacks

early_stopping = callbacks.EarlyStopping(monitor='val_loss',
                                         patience=3,
                                         mode='min',
                                         restore_best_weights=True)

import os
cwd = os.getcwd()
filepath = cwd + '\\out\\concat_RNN_VGG16.hdf5'

checkpoint = callbacks.ModelCheckpoint(filepath=filepath, 
                                       monitor='val_loss',
                                       save_best_only=True,
                                       save_weights_only=False,
                                       mode='min',
                                       save_freq='epoch')

In [ ]:
#model.fit_generator(gen, steps_per_epoch=int(len(y_train.values)/16), validation_data = gentest, validation_steps = int(len(y_test.values)/16),epochs=10, workers=-1,callbacks=[early_stopping, checkpoint])
#model.fit_generator(train_set, steps_per_epoch=int(len(y_train.values)/32), validation_data = test_set, validation_steps = int(len(y_test.values)/32),epochs=10, workers=1)
history = model.fit(train_set, steps_per_epoch=int(len(y_train.values)/16), validation_data = test_set, validation_steps = int(len(y_test.values)/16),epochs=10, workers=-1,callbacks=[early_stopping, checkpoint])

#https://www.geeksforgeeks.org/keras-fit-and-keras-fit_generator/
#So, we have learned the difference between Keras.fit and Keras.fit_generator functions used to train a deep learning neural network
#.fit is used when the entire training dataset can fit into the memory and no data augmentation is applied.
#.fit_generator is used when either we have a huge dataset to fit into our memory or when data augmentation needs to be applied.

Epoch 1/10
2122/2122 [==============================] - ETA: 0s - loss: 2.3048 - accuracy: 0.3282

In [20]:
filepath

'C:\\Users\\Edgar\\Documents\\Rakuten\\out\\concat_RNN_VGG16.hdf5'